# Variables

## Import libraries

In [1]:
import os
import sys
from rdflib import URIRef, Literal

In [2]:
# Existing files
ont_file_name = "ontology.ttl"
ruleset_file_name = "rules.pie"

# Created files during process (in `tmp_folder`)
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"
implicit_to_facts_ttl_file_name = "implicit_to_facts.ttl"

# Existing folders
data_folder_name = "../data"

# Created folder during process
tmp_folder_name = "../tmp_files"

# GraphDB repository name
facts_repository_name = "addresses_from_factoids"

# Definition of names of named graphes 
ontology_named_graph_name = "ontology"
facts_named_graph_name = "facts"
factoids_named_graph_name = "factoids"
permanent_named_graph_name = "permanent"
tmp_named_graph_name = "temporary"
inter_sources_name_graph_name = "inter_sources"

# URIs to access to GraphDB
str_graphdb_url = "http://localhost:7200"

py_code_folder_path = "./code"

In [3]:
tmp_folder = os.path.abspath(tmp_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)
implicit_to_facts_ttl_file = os.path.join(tmp_folder, implicit_to_facts_ttl_file_name)

graphdb_url = URIRef(str_graphdb_url)

In [4]:
# Calling up the `code` folder contains the python codes
sys.path.insert(1, python_code_folder)

import file_management as fm
import graphdb as gd
import graphrdf as gr
import attribute_version_comparisons as avc
import multi_sources_processing as msp
import factoids_creation as fc
import time_processing as tp
import resource_transfert as rt
import evolution_construction as ec
import states_events_json as sej
import create_factoids_descriptions as cfd

In [5]:
# lm_states_descriptions = [
#     {
#         "id": 1,
#         "label": "23",
#         "comment": "23 rue du Père-Guérin",
#         "lang": "fr",
#         "type": "thoroughfare",
#         "attributes": {
#             "geometry": {
#                 "value": "POINT(2.353477 48.830089)",
#                 "data_type": "wkt_literal"
#             },
#             "name": {
#                 "value": "23",
#                 "lang": "fr",
#             }
#         },
#         "time": {
#             "start": {
#                 "stamp": "2020-01-01",
#                 "calendar": "gregorian",
#                 "precision": "day"
#             },
#             "end": {
#                 "stamp": "2025-03-28",
#                 "calendar": "gregorian",
#                 "precision": "day"
#             }
#         },
#         "provenance": {
#             "uri": "https://www.openstreetmap.org/node/2268836757"
#         }
#     },
#     {
#         "id": 2,
#         "label": "rue du Père-Guérin",
#         "lang": "fr",
#         "type": "thoroughfare",
#         "attributes": {
#             "geometry": {
#                 "value": "POLYGON ((2.354497 48.830479, 2.354387 48.830345, 2.35436 48.830364, 2.354296 48.830344, 2.35426 48.830333, 2.354226 48.830323, 2.354103 48.830284, 2.354057 48.83027, 2.353988 48.830248, 2.35394 48.830233, 2.353914 48.830225, 2.353901 48.830221, 2.353887 48.830217, 2.353827 48.830198, 2.353715 48.830163, 2.353557 48.830115, 2.353551 48.830113, 2.353519 48.830103, 2.353359 48.830053, 2.353322 48.830041, 2.353292 48.830032, 2.353239 48.830016, 2.353165 48.829993, 2.353019 48.829947, 2.352937 48.829922, 2.352882 48.829905, 2.352861 48.829881, 2.352857 48.829996, 2.352894 48.829982, 2.353012 48.830018, 2.353201 48.830077, 2.353325 48.830116, 2.353541 48.830181, 2.353583 48.830195, 2.353703 48.830233, 2.353785 48.830258, 2.353937 48.830305, 2.35399 48.830322, 2.354081 48.83035, 2.354248 48.830402, 2.354366 48.830438, 2.354497 48.830479))",
#                 "data_type": "wkt_literal"
#             },
#             "name": {
#                 "value": "rue du Père-Guérin",
#                 "lang": "fr",
#             }
#         },
#         "time": {
#             "start": {
#                 "stamp": "2020-01-01",
#                 "calendar": "gregorian",
#                 "precision": "day"
#             },
#             "end": {
#                 "stamp": "2025-03-28",
#                 "calendar": "gregorian",
#                 "precision": "day"
#             }
#         },
#         "provenance": {
#             "uri": "https://fr.wikipedia.org/wiki/Rue_du_P%C3%A8re-Gu%C3%A9rin"
#         }
#     }
# ]

# lr_states_descriptions = [
#     {
#         "type":"belongs",
#         "locatum":1,
#         "relatum":2,
#         "time": {
#             "start": {
#                 "stamp": "2020-01-01",
#                 "calendar": "gregorian",
#                 "precision": "day"
#             },
#             "end": {
#                 "stamp": "2025-03-28",
#                 "calendar": "gregorian",
#                 "precision": "day"
#             }
#         },
#         "provenance": {
#             "uri": "https://www.openstreetmap.org/node/2268836757"
#         }
#     }
# ]

# description = {"landmarks": lm_states_descriptions, "relations": lr_states_descriptions}

# g = sej.create_graph_from_states_descriptions(description)
# out_file = "/home/CBernard2/Projects/pegazus-extension/tmp_files/test_version.ttl"
# g.serialize(destination=out_file, format="turtle")

In [6]:
# source_valid_time = {
#     "start": {
#         "stamp": "2020-01-01",
#         "calendar": "gregorian",
#         "precision": "day"
#     },
#     "end": {
#         "stamp": "2025-03-28",
#         "calendar": "gregorian",
#         "precision": "day"
#     }
# }
# ban_file_name = "ban_adresses.csv"
# out_file_name = "test_version.ttl"
# data_folder_name = "../data"
# tmp_folder_name = "../tmp_files"

# bpa_file = os.path.join(data_folder, ban_file_name)
# out_file = os.path.join(tmp_folder, out_file_name)

# g = fc.create_graph_from_paris_ban(bpa_file, source_valid_time, "fr")
# g.serialize(destination=out_file, format="turtle")

In [7]:
# source_valid_time = {
#     "start": {
#         "stamp": "2020-01-01",
#         "calendar": "gregorian",
#         "precision": "day"
#     },
#     "end": {
#         "stamp": "2025-03-28",
#         "calendar": "gregorian",
#         "precision": "day"
#     }
# }
# osm_file_name = "osm_adresses.csv"
# osm_hn_file_name = "osm_hn_adresses.csv"
# out_file_name = "test_version.ttl"
# data_folder_name = "../data"
# tmp_folder_name = "../tmp_files"

# osm_file = os.path.join(data_folder, osm_file_name)
# osm_hn_file = os.path.join(data_folder, osm_hn_file_name)
# out_file = os.path.join(tmp_folder, out_file_name)

# # cfd.create_state_description_for_osm(osm_file, osm_hn_file, source_valid_time, "fr", None)

# g = fc.create_graph_from_osm(osm_file, osm_hn_file, source_valid_time, "fr")
# g.serialize(destination=out_file, format="turtle")

In [8]:
# source_valid_time = {
#     "start": {
#         "stamp": "2020-01-01",
#         "calendar": "gregorian",
#         "precision": "day"
#     },
#     "end": {
#         "stamp": "2025-03-28",
#         "calendar": "gregorian",
#         "precision": "day"
#     }
# }
# vpa_file_name = "denominations-emprises-voies-actuelles.csv"
# vpc_file_name = "denominations-des-voies-caduques.csv"
# out_file_name = "test_version.ttl"
# data_folder_name = "../data"
# tmp_folder_name = "../tmp_files"

# vpa_file = os.path.join(data_folder, vpa_file_name)
# vpc_file = os.path.join(data_folder, vpc_file_name)
# out_file = os.path.join(tmp_folder, out_file_name)

# # cfd.create_state_description_for_osm(osm_file, osm_hn_file, source_valid_time, "fr", None)

# g = fc.create_graph_from_ville_paris(vpa_file, vpc_file, source_valid_time, "fr")
# g.serialize(destination=out_file, format="turtle")

In [ ]:
source_valid_time = {
    "start": {
        "stamp": "2020-01-01",
        "calendar": "gregorian",
        "precision": "day"
    },
    "end": {
        "stamp": "2025-03-28",
        "calendar": "gregorian",
        "precision": "day"
    }
}
geojson_file_name = "denominations-emprises-voies-actuelles.csv"
out_file_name = "test_version.ttl"
data_folder_name = "../data"
tmp_folder_name = "../tmp_files"

# geojson_file = os.path.join(data_folder, geojson_file_name)
geojson_file = "/home/CBernard2/Téléchargements/donnees/paris_5/jacoubet_axis_paris_5.geojson"
out_file = os.path.join(tmp_folder, out_file_name)

name_attribute = "historical_name"
g = fc.create_graph_from_geojson_states_of_thoroughfares(geojson_file, 'fr', source_valid_time, "normalised_name", name_attribute)
g.serialize(destination=out_file, format="turtle")


geojson_file = "/home/CBernard2/Téléchargements/donnees/paris_5/jacoubet_addresss_points_paris_5.geojson"
out_file = os.path.join(tmp_folder, "jacoubet_addresss_points_paris_5.ttl")
g = fc.create_graph_from_geojson_states_of_streetnumbers(geojson_file, 'fr', source_valid_time, "historical_name", name_attribute)
g.serialize(destination=out_file, format="turtle")

geojson_file = "/home/CBernard2/Téléchargements/donnees/paris_5/cadastre_general_paris_1825_numbers_initial_import_paris_5.geojson"
out_file = os.path.join(tmp_folder, "cadastre_general_paris_1825_numbers_initial_import_paris_5.ttl")
g = fc.create_graph_from_geojson_states_of_streetnumbers(geojson_file, 'fr', source_valid_time, "historical_name", name_attribute)
g.serialize(destination=out_file, format="turtle")


{'landmarks': [{'id': '94d6924a5fb94b0eb3e26be3a6186d97', 'label': '2', 'type': 'street_number', 'lang': None, 'attributes': {'name': {'value': '2'}, 'geometry': {'value': '<http://www.opengis.net/def/crs/EPSG/0/2154> POINT (652771.146325 6860171.489028)', 'datatype': 'wkt_literal'}}, 'provenance': {}, 'time': {'start': {'stamp': '2020-01-01', 'calendar': 'gregorian', 'precision': 'day'}, 'end': {'stamp': '2025-03-28', 'calendar': 'gregorian', 'precision': 'day'}}}, {'id': '47e3d4493f0841fca3b9ae9cfe880922', 'label': 'Rue du Jardin du Roi', 'type': 'thoroughfare', 'lang': 'fr', 'attributes': {'name': {'value': '2'}, 'geometry': {'value': '<http://www.opengis.net/def/crs/EPSG/0/2154> POINT (652771.146325 6860171.489028)', 'datatype': 'wkt_literal'}}, 'provenance': {}, 'time': {'start': {'stamp': '2020-01-01', 'calendar': 'gregorian', 'precision': 'day'}, 'end': {'stamp': '2025-03-28', 'calendar': 'gregorian', 'precision': 'day'}}}, {'id': 'b0c948090b38444e864376000fd1b5aa', 'label': '4'

<Graph identifier=N52b001f7dd964f1590baf357dd159bd5 (<class 'rdflib.graph.Graph'>)>